In [3]:
import sys, signal
from PyQt4 import QtCore, QtGui, QtWebKit

class WebPage(QtWebKit.QWebPage):  
    def __init__(self, db):
        self._db = db
        QtWebKit.QWebPage.__init__(self)
        self.mainFrame().loadFinished.connect(self.handleLoadFinished)

    def process(self, items):
        self._items = iter(items)
        self.fetchNext()
    
    def fetchNext(self):
        try:
            self._url, self._func = next(self._items)
            self.mainFrame().load(QtCore.QUrl(self._url))
        except StopIteration:
            return False
        return True
    
    def handleLoadFinished(self):
        self._func(self._url, self.mainFrame().toHtml())
        if not self.fetchNext():
            print('# processing complete')
            QtGui.qApp.quit()
            
def funcA(url, html):
    print('# processing:', url)
    # do stuff with soup
    
def funcB(url, html):
    print('# processing:', url)
    # do stuff with soup
    

In [4]:
# load the list of editions conveniently saved as json

import json

load_path = "data/ploscompbio_editions.json"

with open(load_path, 'r') as infile:  
    list_of_editions = json.load(infile)
    
print(list_of_editions)

IOError: [Errno 2] No such file or directory: 'data/ploscompbio_editions.json'

In [ ]:
def process_url(url,html):
    print(url)
    print('testing processing pipeline')
    

In [ ]:
# initialize database

import sqlite3

db = sqlite3.connect('data/articles2.db')
cursor = db.cursor()
cursor.execute('''CREATE TABLE articles2(id INTEGER PRIMARYKEY,
                    url TEXT, authors TEXT, date TEXT, title TEXT, abstract TEXT, author_summary TEXT)
                    ''')


db.commit()


In [ ]:
edition_urls = [ed['url'] for ed in list_of_editions]
edition_urls = edition_urls

items = [ (url,process_url) for url in edition_urls]

signal.signal(signal.SIGINT, signal.SIG_DFL)
print('press ctrl+c to quit\n')
app = QtGui.QApplication(sys.argv)
webpage = WebPage(db)
webpage.process(items)
sys.exit(app.exec_())
